# Multimodal Emotion Recognition from Video Clips

Importing Libraries

In [1]:
from time import time
import cv2
import numpy as np
import glob
import os
from datetime import date, datetime
import pandas

#### LBP Feature Extraction Class with 8 neighbour points and 1 radius
#### Total Bins for Histogram = 27

In [16]:
from skimage import feature
import numpy as np
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
    def describe(self, image, eps=1e-7):
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        # return the histogram of Local Binary Patterns
        return hist

#### initialising the LBP class Object

In [17]:
desc = LocalBinaryPatterns(24, 8)


#### find LBPFeatureVector Function 
#### parameter : path of image 
##### working: - calculate lbp histogram for each frame in a video clip and return avg lbp histogram for the video

In [19]:
def findLBPFeatureVector(path):
    hist = []
    cnt=0
    vidObj = cv2.VideoCapture(path);
    while (vidObj.isOpened()):
        ret,image = vidObj.read()
        if(ret==False):
            break
        image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        curHist = desc.describe(image)
        hist.append(curHist)
        cnt+=1
    vidObj.release();
    return np.average(np.array(hist),axis = 0)

    

####  Iterate over each video clip (from 1-20) actors and store the lbp feature vector (histogram) in data and it's label in label 
#### audio-video folder contains videos of 1-20 actors


In [ ]:
import glob
emotion = {1:'neutral',2:'calm',3:'happy',4:'sad',5:'angry',6:'fearful',7:'disgust',8:'surprise'}
print(emotion)
data = []
label = []
for i in range(1,9):
    files = glob.glob('audio-video\\**\\02-01-0{emotion}-01-01-01-*.mp4'.format(emotion = i))
    for file in files:
        print(file)
        hist = findLBPFeatureVector(file)
        data.append(hist)
        label.append(i)


{1: 'neutral', 2: 'calm', 3: 'happy', 4: 'sad', 5: 'angry', 6: 'fearful', 7: 'disgust', 8: 'surprise'}
audio-video\Actor_01\02-01-01-01-01-01-01.mp4
audio-video\Actor_02\02-01-01-01-01-01-02.mp4
audio-video\Actor_03\02-01-01-01-01-01-03.mp4
audio-video\Actor_04\02-01-01-01-01-01-04.mp4
audio-video\Actor_05\02-01-01-01-01-01-05.mp4
audio-video\Actor_06\02-01-01-01-01-01-06.mp4
audio-video\Actor_07\02-01-01-01-01-01-07.mp4
audio-video\Actor_08\02-01-01-01-01-01-08.mp4
audio-video\Actor_09\02-01-01-01-01-01-09.mp4
audio-video\Actor_10\02-01-01-01-01-01-10.mp4
audio-video\Actor_11\02-01-01-01-01-01-11.mp4
audio-video\Actor_12\02-01-01-01-01-01-12.mp4
audio-video\Actor_13\02-01-01-01-01-01-13.mp4
audio-video\Actor_14\02-01-01-01-01-01-14.mp4
audio-video\Actor_15\02-01-01-01-01-01-15.mp4
audio-video\Actor_16\02-01-01-01-01-01-16.mp4


In [ ]:
data = np.array(data)
label = np.array(label)
print(data.shape)
print(label.shape)

#### Linear Support Vector Classifier used for training the model and predicting the emotion of test video

In [ ]:
from sklearn.svm import LinearSVC
print(len(label))
# print(data)
# train a Linear SVM on the data
model = LinearSVC(random_state=42)
model.fit(data, label)

   

#### Saving and loading the model using pickle

In [ ]:
import pickle
with open('finalTrainedModelv2_pkl', 'wb') as files:
    pickle.dump(model, files)

In [ ]:
with open('finalTrainedModelv2_pkl' , 'rb') as f:
    lr = pickle.load(f)

display(lr)

#### testing the model on 21-24 actors

In [ ]:
test_data=[]
test_true_label = []
for i in range(1,9):
    test_files = []
    test_files = glob.glob('test\\**\\02-01-0{emotion}-01-01-01-*.mp4'.format(emotion = i))
    for file in test_files:
        print(file)
        hist = findLBPFeatureVector(file)
        test_data.append(hist)
        test_true_label.append(i)

In [ ]:
test_true_label = np.array(test_true_label)
test_predict_label = np.array(model.predict(test_data))
display(test_predict_label.shape)

In [ ]:

print(test_predict_label)
print(test_true_label.shape)
print(test_true_label)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(test_true_label,test_predict_label))

              precision    recall  f1-score   support

           1       0.17      0.25      0.20         4
           2       0.12      0.50      0.20         4
           3       0.12      0.25      0.17         4
           4       0.00      0.00      0.00         4
           5       0.50      0.25      0.33         4
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00         4

    accuracy                           0.16        32
   macro avg       0.11      0.16      0.11        32
weighted avg       0.11      0.16      0.11        32



C:\Users\vishal bansal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vishal bansal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vishal bansal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi